# Prova 3
## Parte I - Programação Inteira
Uma empresa opera 2 aeronaves de pequeno porte (A1 e A2) e precisa escalar tripulação para cumprir uma rota diária de ida e volta em dois trechos por aeronave:

- A1: Cidade Juiz de Fora → Rio → Juiz de Fora (ida e volta no mesmo dia)
- A2: Cidade Juiz de Fora → Brasília → Juiz de Fora (ida e volta no mesmo dia)

O planejamento é para T=4 (D1…D4).

Em cada dia, cada aeronave que voar precisa de:

- 1 piloto
- 1 comissário(a)

**Regras trabalhistas simplificadas:**

- Cada tripulante não pode trabalhar em dias consecutivos  (descanso mínimo de 1 dia entre trabalhos).
- Se trabalhou no dia t, não pode trabalhar no dia t+1.
- Um tripulante não pode estar em duas aeronaves no mesmo dia.
- Você pode optar por  cancelar o voo de uma aeronave em determinado dia, pagando uma  penalidade (para manter o problema sempre factível, mesmo com poucas pessoas).
- Cada piloto/comissário tem um custo diário ao trabalhar.

**Objetivo: minimizar o custo total de tripulação + penalidades de cancelamento.**

**Dados da instância - Dias**
- $t ∈ {1,2,3,4}$

**Aeronaves**
- $a ∈ {A1,A2}$

**Pilotos**

| Piloto | Custo/dia |
|:------:|:---------:|
|   P1   |    520    |
|   P2   |    480    |
|   P3   |    450    |

**Comissãrios**

| Comissário | Custo/dia |
|:------:|:---------:|
|   C1   |    260    |
|   C2   |    240    |
|   C3   |    220    |

**Penalidade de Cancelamento**
Cancelar A1 em um dia: 800
Cancelar A2 em um dia: 800

Com essa penalidade, o modelo normalmente faz com que as aeronaves voem todos os dias, mas pode cancelar se o descanso tornar impossível ou caro demais.  

Observação: Entregar apenas a parte de programação inteira com PYSCIPOPT até as 17 horas.


# Formulação do Problema



## Conjuntos do problema:
- $T = \{1, 2, 3, 4\}$: Conjunto de dias.

- $A = \{A1, A2\}$: Conjunto de aeronaves.

- $P = \{P1, P2, P3\}$: Conjunto de pilotos disponíveis.

- $C = \{C1, C2, C3\}$: Conjunto de comissários disponíveis.



## Custos:
- $CostP_p = \{C_{P1}, C_{P2}, C_{P3}\}$: Custo diário do piloto
- $CostC_c = \{C_{C1}, C_{C2}, C_{C3}\}$: Custo diário do comissário
- $M = 800$: Penalidade por cancelar um voo.



## Variáveis de decisão:
- Variável de pilotos($x$):
$$x_{p,a,t} \in \{0, 1\}$$
  - Recebe valor 1 se o piloto $p$ for alocado na aeronave $a$ no dia $t$.
  - Recebe valor 0 caso contrário.

- Variável de comissário($y$):
$$y_{c,a,t} \in \{0, 1\}$$
  - Recebe valor 1 se o comissário $c$ for alocado na aeronave $a$ no dia $t$.
  - Recebe valor 0 caso contrário.

- Variável de multa($z$):
$$z_{a,t} \in \{0, 1\}$$
  - Recebe valor 1 se o voo $a$ for realizado no dia $t$.
  - Recebe valor 0 caso for cancelado.



## FOB: Minimizar o custo total
  1. Pagamento dos pilotos escalados.
  2. Pagamento dos comissários escalados.
  3. Custo de cancelamento

ex: para cada dia:
$$Z_{\text{dia t}} = \underbrace{\text{Custos Pilotos}}_{\text{Se x=1, paga}} + \underbrace{\text{Custos Comissários}}_{\text{Se y=1, paga}} + \underbrace{\text{Multa Cancelamento}}_{\text{Se z=1, paga}}$$

Generalizado:

$$\text{Min } Z = \sum_{t \in T} \sum_{a \in A} \sum_{p \in P} (CostP_p \cdot x_{p,a,t}) + \sum_{t \in T} \sum_{a \in A} \sum_{c \in C} (CostC_c \cdot y_{c,a,t}) + \sum_{t \in T} \sum_{a \in A} M \cdot (1 - z_{a,t})$$



## Restrições:

- Se $z_{a,t}=1$ (voo não cancelado) precisa de 1 piloto e 1 comissário. se $z_{a,t}=0$, ninguém é alocado.

Para todo dia $t \in T$ e aeronave $a \in A$:

$$\sum_{p \in P} x_{p,a,t} = z_{a,t}$$

$$\sum_{c \in C} y_{c,a,t} = z_{a,t}$$

- Regra dos tripulantes
  1. Se trabalha no dia t, não trabalha no dia t+1.
  2. Não pode estar na A1 e A2 no mesmo dia.

Para todo tripulante $(p,c) \in (P,C)$ e dias $t = 1, 2, 3$ (até o penúltimo dia):

$$\underbrace{\sum_{a \in A} (x,y)_{(p,c),a,t}}_{\text{Trabalho hoje}} + \underbrace{\sum_{a \in A} (x,y)_{(p,c),a,t+1}}_{\text{Trabalho amanhã}} \leq 1$$

Obs: Para o último dia ($t=4$), basta garantir que ele não pegue duas aeronaves:

$$\sum_{a \in A} (x,y)_{(p,c),a,4} \leq 1$$




In [2]:
!pip install pyscipopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 42.0 MB/s eta 0:00:00


In [13]:
from pyscipopt import Model, quicksum

## Definições do Problema
# Conjuntos
dias = [1, 2, 3, 4]
aeronaves = ['A1', 'A2']

# Pilotos e Custos
pilotos = ['P1', 'P2', 'P3']
custo_pilotos = {'P1': 520, 'P2': 480, 'P3': 450}

# Comissários e Custos
comissarios = ['C1', 'C2', 'C3']
custo_comissarios = {'C1': 260, 'C2': 240, 'C3': 220}

# Multa por cancelamento
multa_cancelamento = 800

model = Model("Escala_Tripulação")

## Variáveis de decisão
# x[p, a, d] = {0, 1:escalado} piloto p, aeronave a, dia d
x = {}
for p in pilotos:
    for a in aeronaves:
        for d in dias:
            x[p, a, d] = model.addVar(vtype="B", name=f"x_{p}_{a}_{d}")

# y[c, a, d] = {0, 1:escalado} comissario c, aeronave a, dia d
y = {}
for c in comissarios:
    for a in aeronaves:
        for d in dias:
            y[c, a, d] = model.addVar(vtype="B", name=f"y_{c}_{a}_{d}")

# z[a, d] = {0, 1:acontece} aviao a, dia d
z = {}
for a in aeronaves:
    for d in dias:
        z[a, d] = model.addVar(vtype="B", name=f"z_{a}_{d}")


## Restrições

# R1: Escalação do voo
# se z=1, precisa 1 piloto e 1 comissário; se z=0, não tem tripulação
for d in dias:
    for a in aeronaves:
        # Soma dos pilotos alocados == z[a,d]
        model.addCons(quicksum(x[p, a, d] for p in pilotos) == z[a, d], name=f"Req_Piloto_{a}_{d}")
        # Soma dos comissáros alocados == z[a,d]
        model.addCons(quicksum(y[c, a, d] for c in comissarios) == z[a, d], name=f"Req_Comiss_{a}_{d}")

# R2: Descanso e Unicidade Pilotos
for p in pilotos:
    # Descanso: Se trabalhar dia t, não trabalha t+1 (dias 1 a 3)
    for d in range(1, 4):
        model.addCons(
            quicksum(x[p, a, d] for a in aeronaves) +
            quicksum(x[p, a, d+1] for a in aeronaves) <= 1,
            name=f"Descanso_Piloto_{p}_d{d}"
        )

    # Unicidade: Não voar 2 aeronaves no mesmo dia (Para TODOS os dias)
    for d in dias:
        model.addCons(quicksum(x[p, a, d] for a in aeronaves) <= 1, name=f"Unicidade_Piloto_{p}_d{d}")

# R3: Descanso e Unicidade Comissários
for c in comissarios:
    # Descanso
    for d in range(1, 4):
        model.addCons(
            quicksum(y[c, a, d] for a in aeronaves) +
            quicksum(y[c, a, d+1] for a in aeronaves) <= 1,
            name=f"Descanso_Comiss_{c}_d{d}"
        )

    # Unicidade
    for d in dias:
        model.addCons(quicksum(y[c, a, d] for a in aeronaves) <= 1, name=f"Unicidade_Comiss_{c}_d{d}")


## Função objetivo
# Minimizar: Custo Pilotos + Custo Comissários + Penalidades
obj_pilotos = quicksum(custo_pilotos[p] * x[p, a, d] for p in pilotos for a in aeronaves for d in dias)
obj_comissarios = quicksum(custo_comissarios[c] * y[c, a, d] for c in comissarios for a in aeronaves for d in dias)

# Penalidade: Custo Total (se tudo fosse cancelado) - Economia dos voos realizados
total_penalidades_possiveis = multa_cancelamento * len(aeronaves) * len(dias)
obj_economias_voo = quicksum(multa_cancelamento * z[a, d] for a in aeronaves for d in dias)

model.setObjective(obj_pilotos + obj_comissarios + total_penalidades_possiveis - obj_economias_voo, "minimize")


## Solução:
model.optimize()

## Impressão:
status = model.getStatus()

if status == "optimal":
    print("\n=== Solução Ótima Encontrada ===")
    print(f"Custo Total Mínimo: R$ {model.getObjVal():.2f}")
    print("-" * 60)
    print(f"{'Dia':<5} | {'Aeronave':<10} | {'Status':<10} | {'Piloto':<10} | {'Comissário':<10}")
    print("-" * 60)

    dias_ordenados = sorted(dias)
    aeronaves_ordenadas = sorted(aeronaves)

    for d in dias_ordenados:
        for a in aeronaves_ordenadas:
            voo_realizado = model.getVal(z[a, d]) > 0.5
            if voo_realizado:
                p_escalado = ""
                for p in pilotos:
                    if model.getVal(x[p, a, d]) > 0.5:
                        p_escalado = f"{p} ({custo_pilotos[p]})"
                        break

                c_escalado = ""
                for c in comissarios:
                    if model.getVal(y[c, a, d]) > 0.5:
                        c_escalado = f"{c} ({custo_comissarios[c]})"
                        break

                print(f"D{d:<4} | {a:<10} | {'Confirmado':<10} | {p_escalado:<10} | {c_escalado:<10}")
            else:
                print(f"D{d:<4} | {a:<10} | {'CANCELADO':<10} | {'-':<10} | {'-':<10}")
        print("-" * 60)
else:
    print("Não foi encontrada solução ótima.")


=== Solução Ótima Encontrada ===
Custo Total Mínimo: R$ 5940.00
------------------------------------------------------------
Dia   | Aeronave   | Status     | Piloto     | Comissário
------------------------------------------------------------
D1    | A1         | Confirmado | P3 (450)   | C1 (260)  
D1    | A2         | Confirmado | P1 (520)   | C3 (220)  
------------------------------------------------------------
D2    | A1         | Confirmado | P2 (480)   | C2 (240)  
D2    | A2         | CANCELADO  | -          | -         
------------------------------------------------------------
D3    | A1         | Confirmado | P3 (450)   | C1 (260)  
D3    | A2         | Confirmado | P1 (520)   | C3 (220)  
------------------------------------------------------------
D4    | A1         | Confirmado | P2 (480)   | C2 (240)  
D4    | A2         | CANCELADO  | -          | -         
------------------------------------------------------------
